<a href="https://colab.research.google.com/github/tasnimislamraisa/Python_Learning/blob/deep-Learning/with_multiple_speaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install gdown
import gdown
import zipfile
import os

# Download ZIP from shared link
file_id = "1x5Fm1vsuAo8AvESLsywU5NuCKGmiIj9I"
url = f"https://drive.google.com/uc?id={file_id}"
output = "grid_dataset.zip"
gdown.download(url, output, quiet=False)

# Extract to /content/data
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("/content/data")

# Confirm structure
for root, dirs, files in os.walk("/content/data"):
    print("📁", root)
    for d in dirs:
        print("   └──", d)


Downloading...
From (original): https://drive.google.com/uc?id=1x5Fm1vsuAo8AvESLsywU5NuCKGmiIj9I
From (redirected): https://drive.google.com/uc?id=1x5Fm1vsuAo8AvESLsywU5NuCKGmiIj9I&confirm=t&uuid=9f75ecc7-6455-47e7-a68e-a3c2117a8b91
To: /content/grid_dataset.zip
100%|██████████| 13.6G/13.6G [02:46<00:00, 81.4MB/s]


📁 /content/data
   └── data
📁 /content/data/data
   └── s15_processed
   └── s27_processed
   └── s31_processed
   └── s2_processed
   └── s19_processed
   └── s5_processed
   └── s9_processed
   └── s33_processed
   └── s24_processed
   └── s8_processed
   └── s1_processed
   └── s25_processed
   └── s12_processed
   └── s32_processed
   └── s20_processed
   └── s30_processed
   └── s13_processed
   └── s23_processed
   └── s34_processed
   └── s7_processed
   └── s3_processed
   └── s10_processed
   └── s6_processed
   └── s14_processed
   └── s16_processed
   └── s22_processed
   └── s28_processed
   └── s26_processed
   └── s4_processed
   └── s11_processed
   └── s29_processed
   └── s18_processed
   └── s17_processed
📁 /content/data/data/s15_processed
   └── align
📁 /content/data/data/s15_processed/align
📁 /content/data/data/s27_processed
   └── align
📁 /content/data/data/s27_processed/align
📁 /content/data/data/s31_processed
   └── align
📁 /content/data/data/s31_processed/align


# **🔹 1. Install Dependencies**

In [ ]:
!pip install editdistance
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


# **🔹 2. Import Libraries & Define Vocabulary**

In [ ]:
import os, cv2
import numpy as np
import tensorflow as tf
import editdistance
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from matplotlib import pyplot as plt

In [ ]:
# Vocabulary
vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True)


# **🔹 3. Preprocess & Save NPZ from 30 Speakers × 75 Samples**

In [ ]:
def preprocess_all(base_path='/content/data/data', out_dir='./npz_data', speakers=range(1, 31), samples_per_speaker=30):
    os.makedirs(out_dir, exist_ok=True)
    count = 0

    for s in speakers:
        spk = f"s{s}_processed"
        v_path = os.path.join(base_path, spk)
        a_path = os.path.join(v_path, "align")

        files = sorted([f.split('.')[0] for f in os.listdir(v_path) if f.endswith('.mpg')])[:samples_per_speaker]
        for fname in files:
            video_file = os.path.join(v_path, fname + ".mpg")
            align_file = os.path.join(a_path, fname + ".align")

            if not os.path.exists(video_file) or not os.path.exists(align_file): continue

            cap = cv2.VideoCapture(video_file)
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret: break
                frame = tf.image.rgb_to_grayscale(frame)
                frames.append(frame[190:236, 80:220, :])
            cap.release()

            frames = tf.cast(frames, tf.float32)
            frames = (frames - tf.math.reduce_mean(frames)) / tf.math.reduce_std(frames)

            with open(align_file, 'r') as f:
                lines = f.readlines()
            tokens = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) == 3 and parts[2] != 'sil':
                    tokens.extend([' ', parts[2]])
            labels = char_to_num(tf.reshape(tf.strings.unicode_split(tokens, 'UTF-8'), (-1)))[1:]

            np.savez_compressed(os.path.join(out_dir, f"{spk}_{fname}.npz"),
                                frames=frames.numpy(), alignments=labels.numpy())
            count += 1

    print(f"✅ Saved {count} preprocessed samples")


In [ ]:
import os

base_path = '/content/data/data'
all_folders = os.listdir(base_path)
available_speakers = sorted([f for f in all_folders if f.endswith('_processed')])
print(f"✅ Found {len(available_speakers)} speakers")
print(available_speakers)


✅ Found 33 speakers
['s10_processed', 's11_processed', 's12_processed', 's13_processed', 's14_processed', 's15_processed', 's16_processed', 's17_processed', 's18_processed', 's19_processed', 's1_processed', 's20_processed', 's22_processed', 's23_processed', 's24_processed', 's25_processed', 's26_processed', 's27_processed', 's28_processed', 's29_processed', 's2_processed', 's30_processed', 's31_processed', 's32_processed', 's33_processed', 's34_processed', 's3_processed', 's4_processed', 's5_processed', 's6_processed', 's7_processed', 's8_processed', 's9_processed']


In [ ]:
import re

# ✅ Match only folders like 's10_processed'
pattern = re.compile(r"s(\d+)_processed")
speakers = []

for s in available_speakers:
    match = pattern.fullmatch(s)
    if match:
        speakers.append(int(match.group(1)))

print("✅ Valid speakers found:", speakers)


✅ Valid speakers found: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 22, 23, 24, 25, 26, 27, 28, 29, 2, 30, 31, 32, 33, 34, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
preprocess_all(base_path=base_path, speakers=speakers, samples_per_speaker=30)


✅ Saved 990 preprocessed samples


# **🔹 4. Dataset Loader**

In [ ]:
def load_npz(file_path):
    with np.load(file_path) as data:
        return data['frames'], data['alignments']

def tf_data_wrapper(file_path):
    def _load(file):
        f = file.numpy().decode()
        frames, alignments = load_npz(f)
        return frames, alignments
    video, labels = tf.py_function(_load, [file_path], (tf.float32, tf.int64))
    video.set_shape([None, 46, 140, 1])
    labels.set_shape([None])
    return video, labels

def get_datasets(npz_dir='./npz_data', batch_size=8, val_split=0.2):
    files = sorted(tf.io.gfile.glob(f"{npz_dir}/*.npz"))
    split_idx = int(len(files) * (1 - val_split))
    train_files = tf.data.Dataset.from_tensor_slices(files[:split_idx])
    val_files = tf.data.Dataset.from_tensor_slices(files[split_idx:])

    def pipeline(file_ds):
        ds = file_ds.map(tf_data_wrapper, num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.cache().shuffle(100).padded_batch(batch_size, padded_shapes=([None, 46, 140, 1], [None]))
        return ds.prefetch(tf.data.AUTOTUNE)

    return pipeline(train_files), pipeline(val_files)


# **🔹 5. Model & Callbacks**

In [ ]:
def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_len = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_len = tf.cast(tf.shape(y_true)[1], dtype="int64")
    input_len = input_len * tf.ones((batch_len, 1), dtype="int64")
    label_len = label_len * tf.ones((batch_len, 1), dtype="int64")
    return tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_len, label_len)


In [ ]:
def build_model():
    inputs = layers.Input(shape=(None, 46, 140, 1))
    x = layers.Conv3D(128, 3, padding='same', activation='relu')(inputs)
    x = layers.MaxPool3D((1, 2, 2))(x)
    x = layers.Conv3D(256, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool3D((1, 2, 2))(x)
    x = layers.Conv3D(75, 3, padding='same', activation='relu')(x)
    x = layers.MaxPool3D((1, 2, 2))(x)
    x = layers.TimeDistributed(layers.Flatten())(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(char_to_num.vocabulary_size() + 1, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss=CTCLoss)
    return model


In [ ]:
def get_callbacks():
    checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
    schedule = LearningRateScheduler(lambda e, lr: lr if e < 30 else lr * tf.math.exp(-0.1).numpy())
    earlystop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    return [checkpoint, schedule, earlystop]

# **🔹 6. Train the Model**

In [ ]:
train_data, val_data = get_datasets(batch_size=8)
model = build_model()
callbacks = get_callbacks()

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
27/99 ━━━━━━━━━━━━━━━━━━━━ 3:56:42 197s/step - loss: 104.2567

# **🔹 7. Evaluate & Visualize Predictions**

In [ ]:
def evaluate_batch(dataset, num_to_char, model):
    total_cer = 0
    count = 0
    for videos, labels in dataset.take(1):
        yhat = model.predict(videos)
        decoded = tf.keras.backend.ctc_decode(yhat, [yhat.shape[1]]*yhat.shape[0], greedy=False, beam_width=10)[0][0]
        for i in range(len(decoded)):
            pred = tf.strings.reduce_join(num_to_char(decoded[i])).numpy().decode()
            actual = tf.strings.reduce_join(num_to_char(labels[i])).numpy().decode()
            cer = editdistance.eval(actual.strip(), pred.strip()) / max(len(actual.strip()), 1)
            print(f"✅ Actual: {actual}\n🔮 Predicted: {pred}\n📏 CER: {cer:.2%}\n{'-'*40}")
            total_cer += cer
            count += 1
    print(f"📊 Average CER: {(total_cer / count):.2%}")

evaluate_batch(val_data, num_to_char, model)


NameError: name 'val_data' is not defined